# Notebook 17: Resolution de Sudoku avec Large Language Models (LLM)

[< Retour a l'index](./README.md) | **Notebook precedent**: [16 - Neural Network](./Sudoku-16-NeuralNetwork.ipynb) | **Notebook suivant**: [18 - Comparison](./Sudoku-18-Comparison.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
- **Comprendre** les capacites et limites des LLM pour la resolution de Sudoku
- **Implementer** differentes approches : Zero-shot, Few-shot, Chain-of-Thought
- **Utiliser** le LLM comme generateur de code (code interpreter)
- **Evaluer** la performance des LLM vs solveurs algorithmiques

**Duree estimee** : 30-40 minutes
**Prerequis** : Aucun, connaissance basique des LLM recommandee
**API requise** : OpenAI API ou compatible

## Introduction : LLM et Resolution de Problemes Combinatoires

### Pourquoi utiliser un LLM pour Sudoku ?

Le Sudoku est un probleme **combinatoire** qui demande :
- **Raisonnement logique** : Deduction et propagation de contraintes
- **Explosion combinatoire** : 9^81 possibilites theoriques
- **Precision** : Une seule erreur invalide toute la solution

Les LLM (GPT-4, Claude, etc.) sont :
- **Entraînes sur du code** : Connaissent les algorithmes de Sudoku
- **Capables de raisonnement** : Peuvent suivre des etapes logiques
- **Generatifs** : Peuvent ecrire du code pour resoudre le probleme

### Approches LLM pour Sudoku

| Approche | Description | Avantages | Inconvenients |
|----------|-------------|-----------|---------------|
| **Zero-shot** | Donner la grille et demander la solution | Simple | Taux d'echec eleve |
| **Few-shot** | Donner des exemples resolus | Meilleure performance | Consomme des tokens |
| **Chain-of-Thought** | Demander au LLM d'expliquer sa démarche | Meilleure raisonnement | Plus lent |
| **Code Interpreter** | Le LLM écrit et execute du code Python | Tres fiable | Necessite execution |

### Performance attendue

Selon les études récentes (2023-2025) :
- **GPT-4** : ~30-50% de réussite en zero-shot sur Sudokus difficiles
- **Claude 3.5 Sonnet** : ~40-60% de réussite
- **Code Interpreter** : ~99% de réussite (genere du code valide)

Les solveurs algorithmiques (backtracking, OR-Tools, Z3) restent **superieurs** en performance et fiabilité.

In [ ]:
# Imports et configuration
import os
import json
import time
from pathlib import Path
from typing import List, Optional, Dict

# Configuration du chemin vers les puzzles
NOTEBOOK_DIR = Path(r"D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku")
PUZZLES_DIR = NOTEBOOK_DIR / "Puzzles"

print(f"Dossier Puzzles: {PUZZLES_DIR}")
print(f"Fichiers disponibles: {[f.name for f in PUZZLES_DIR.glob('*.txt')]}")

In [ ]:
# Fonctions de chargement des puzzles
def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    """Charge les puzzles depuis un fichier."""
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

def puzzle_to_grid(puzzle_str: str) -> List[List[int]]:
    """Convertit une chaîne de 81 caracteres en grille 9x9."""
    return [[int(puzzle_str[i * 9 + j]) if puzzle_str[i * 9 + j] in '123456789' else 0 
             for j in range(9)] for i in range(9)]

def grid_to_puzzle(grid: List[List[int]]) -> str:
    """Convertit une grille 9x9 en chaîne de 81 caracteres."""
    return ''.join(str(grid[i][j]) if grid[i][j] != 0 else '.' 
                   for i in range(9) for j in range(9))

def print_grid(grid: List[List[int]]) -> None:
    """Affiche une grille de Sudoku."""
    for i in range(9):
        if i % 3 == 0 and i > 0:
            print("-" * 21)
        row = ""
        for j in range(9):
            if j % 3 == 0 and j > 0:
                row += "| "
            val = grid[i][j]
            row += str(val) if val != 0 else "."
            row += " "
        print(row)

def validate_solution(grid: List[List[int]], original: List[List[int]]) -> bool:
    """Verifie qu'une solution est valide."""
    # Verifier que les valeurs originales sont preservees
    for i in range(9):
        for j in range(9):
            if original[i][j] != 0 and grid[i][j] != original[i][j]:
                return False
    
    # Verifier les lignes
    for i in range(9):
        if len(set(grid[i])) != 9:
            return False
    
    # Verifier les colonnes
    for j in range(9):
        col = [grid[i][j] for i in range(9)]
        if len(set(col)) != 9:
            return False
    
    # Verifier les blocs 3x3
    for bi in range(3):
        for bj in range(3):
            block = []
            for i in range(3):
                for j in range(3):
                    block.append(grid[bi*3+i][bj*3+j])
            if len(set(block)) != 9:
                return False
    
    return True

# Charger les puzzles
easy_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=5)
hard_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_hardest.txt'))

print(f"Puzzles faciles charges: {len(easy_puzzles)}")
print(f"Puzzles difficiles charges: {len(hard_puzzles)}")

# Afficher un puzzle exemple
example_grid = puzzle_to_grid(easy_puzzles[0])
print("\nExemple de puzzle facile:")
print_grid(example_grid)

## Configuration de l'API LLM

Ce notebook supporte plusieurs providers LLM via OpenAI ou des APIs compatibles.

### Providers supportes

| Provider | Model | Base URL | API Key |
|----------|-------|----------|----------|
| **OpenAI** | gpt-4, gpt-4-turbo | https://api.openai.com/v1 | OPENAI_API_KEY |
| **Anthropic** | claude-3-5-sonnet | Via SDK | ANTHROPIC_API_KEY |
| **OpenRouter** | gpt-4, claude-3, etc. | https://openrouter.ai/api/v1 | OPENROUTER_API_KEY |
| **z.ai** | GLM-5 | https://api.z.ai/api/anthropic | ZAI_API_KEY |

### Configuration des variables d'environnement

Creer un fichier `.env` dans le dossier `GenAI/` avec :
```bash
OPENAI_API_KEY=sk-...
OPENAI_BASE_URL=https://api.openai.com/v1  # Optionnel
```

In [ ]:
# Configuration du client LLM
# Essayer differentes methodes de connexion
import os

class LLMClient:
    """Client generique pour appels LLM."""
    
    def __init__(self, provider="openai", api_key=None, base_url=None):
        self.provider = provider
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.base_url = base_url or os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
        
        if not self.api_key:
            print("ATTENTION: Pas de API key trouvee. Utilisation du mode simulation.")
            self.api_key = "mock"
    
    def call(self, messages: List[Dict], model: str = "gpt-4", max_tokens: int = 2000) -> str:
        """Appelle l'API LLM.
        
        Args:
            messages: Liste de messages {role, content}
            model: Nom du modele
            max_tokens: Tokens maximaux en reponse
            
        Returns:
            Reponse du LLM
        """
        if self.api_key == "mock":
            return self._mock_call(messages)
        
        # Implementation avec OpenAI SDK ou requests
        try:
            import openai
            client = openai.OpenAI(api_key=self.api_key, base_url=self.base_url)
            
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.0  # Deterministe pour Sudoku
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Erreur API: {e}")
            print("Utilisation du mode simulation.")
            return self._mock_call(messages)
    
    def _mock_call(self, messages: List[Dict]) -> str:
        """Simulation pour tests sans API key."""
        user_msg = messages[-1].get("content", "")[:100]
        return f"[Simulation LLM] Reponse pour: {user_msg}..."

# Initialiser le client
client = LLMClient()
print(f"Client LLM initialise: {client.provider}")
print(f"API Key configuree: {client.api_key != 'mock'}")

## Approche 1 : Zero-Shot Prompting

La methode la plus simple consiste a donner la grille au LLM et lui demander de la resoudre directement.

### Prompt Zero-Shot

```
Voici une grille de Sudoku. Les cases vides sont representees par des points (.).
Complete la grille en respectant les regles du Sudoku (chaque ligne, colonne et bloc 3x3
doit contenir les chiffres 1-9 exactement une fois).

Grille:
53..7....
6..195...
.98....6.
8...6...3
4..8.3..1
7...2...6
.6....28.
...419..5
....8..79

Reponds uniquement avec la grille completee, au meme format.
```

### Avantages et Inconvenients

- **Avantages** : Simple, rapide, pas d'exemples necessaires
- **Inconvenients** :
  - Taux d'echec eleve (~50-70%)
  - Erurs frequentes dans les grilles difficiles
  - Le LLM peut "halluciner" des solutions invalides

In [ ]:
def grid_to_string_format(grid: List[List[int]]) -> str:
    """Convertit une grille en format texte pour le prompt."""
    lines = []
    for i in range(9):
        line = ""
        for j in range(9):
            line += str(grid[i][j]) if grid[i][j] != 0 else "."
        lines.append(line)
    return "\n".join(lines)

def parse_llm_response(response: str) -> Optional[List[List[int]]]:
    """Tente d'extraire une grille depuis la reponse du LLM."""
    # Chercher une sequence de 81 chiffres/points
    import re
    
    # Nettoyer la reponse
    cleaned = response.replace(" ", "").replace("\n", "")
    
    # Chercher un motif de 81 caracteres
    match = re.search(r'[^0-9.]*([0-9.]{81})', cleaned)
    if match:
        puzzle_str = match.group(1)
        return puzzle_to_grid(puzzle_str)
    
    return None

def zero_shot_solve(grid: List[List[int]], client: LLMClient) -> Optional[List[List[int]]]:
    """Tente de resoudre un Sudoku avec zero-shot prompting."""
    
    prompt = f"""Voici une grille de Sudoku. Les cases vides sont representees par des points (.).
Complete la grille en respectant les regles du Sudoku (chaque ligne, colonne et bloc 3x3
doit contenir les chiffres 1-9 exactement une fois).

Grille:
{grid_to_string_format(grid)}

Reponds uniquement avec la grille completee, au meme format, sans aucune explication.
"""
    
    response = client.call([
        {"role": "user", "content": prompt}
    ])
    
    solution = parse_llm_response(response)
    return solution

# Test zero-shot
print("=== Test Zero-Shot ===\n")
start = time.time()
solution = zero_shot_solve(example_grid, client)
elapsed = time.time() - start

if solution:
    print("Solution proposee par le LLM:")
    print_grid(solution)
    
    is_valid = validate_solution(solution, example_grid)
    print(f"\nSolution valide: {is_valid}")
    print(f"Temps: {elapsed:.2f} secondes")
else:
    print("Impossible de parser la reponse du LLM.")

## Approche 2 : Few-Shot Prompting

Le few-shot prompting donne au LLM des exemples de puzzles resolus pour improve la performance.

### Prompt Few-Shot

```
Exemple 1:
Input:
53..7....
6..195...
.98....6.
...

Output:
534678912
672195348
198342567
859761423
426853791
713924856
961537284
287419635
345286179

Exemple 2:
...

Maintenant, resous ce puzzle:
[grille]
```

### Avantages et Inconvenients

- **Avantages** :
  - Meilleure comprehension du format attendu
  - Taux de réussite augmente (~60-80%)
- **Inconvenients** :
  - Consomme plus de tokens (exemples)
  - Plus lent
  - Encore limite sur les puzzles difficiles

In [ ]:
# Exemples pour few-shot learning
FEW_SHOT_EXAMPLES = '''Exemple 1:
Input:
53..7....
6..195...
.98....6.
8...6...3
4..8.3..1
7...2...6
.6....28.
...419..5
....8..79

Output:
534678912
672195348
198342567
859761423
426853791
713924856
961537284
287419635
345286179

Exemple 2:
Input:
.....3..
..5.....
.1.697...
....245..
8.......
..1....9.
...7.4..
......87.
.9.....2.

Output:
487513962
925846731
319697258
673124589
852379614
241568397
598732146
164985273
736251825

'''

def few_shot_solve(grid: List[List[int]], client: LLMClient) -> Optional[List[List[int]]]:
    """Tente de resoudre un Sudoku avec few-shot prompting."""
    
    prompt = f"""Tu es un expert en Sudoku. Voici des exemples de puzzles et leurs solutions:

{FEW_SHOT_EXAMPLES}
Maintenant, resous ce puzzle. Reponds uniquement avec la grille completee:
{grid_to_string_format(grid)}
"""
    
    response = client.call([
        {"role": "user", "content": prompt}
    ])
    
    solution = parse_llm_response(response)
    return solution

# Test few-shot
print("=== Test Few-Shot ===\n")
start = time.time()
solution = few_shot_solve(example_grid, client)
elapsed = time.time() - start

if solution:
    print("Solution proposee par le LLM:")
    print_grid(solution)
    
    is_valid = validate_solution(solution, example_grid)
    print(f"\nSolution valide: {is_valid}")
    print(f"Temps: {elapsed:.2f} secondes")
else:
    print("Impossible de parser la reponse du LLM.")

## Approche 3 : Code Interpreter (Generation de Code)

L'approche la plus fiable consiste a demander au LLM de **generer du code Python** qui resout le Sudoku, puis d'executer ce code.

### Prompt Code Interpreter

```
Ecris une fonction Python `solve_sudoku(grid)` qui resout cette grille:
[grille au format liste de listes]

La fonction doit:
1. Utiliser un algorithme de backtracking
2. Retourner la grille resolue ou None
3. Ne pas utiliser de librairies externes

Reponds uniquement avec le code Python, sans explication.
```

### Avantages et Inconvenients

- **Avantages** :
  - Taux de réussite très élevé (~95-99%)
  - Le code genere est verifiable
  - Fonctionne sur tous les types de puzzles
- **Inconvenients** :
  - Necessite d'executer du code (risque de securite)
  - Plus lent (generation + execution)
  - Depend de la capacite du LLM a ecrire du code valide

In [ ]:
def code_interpreter_solve(grid: List[List[int]], client: LLMClient) -> Optional[List[List[int]]]:
    """Utilise le LLM pour generer du code Python qui resout le Sudoku."""
    
    prompt = f"""Ecris une fonction Python `solve_sudoku(grid)` qui resout cette grille de Sudoku:

grid = {grid}

La fonction doit:
1. Utiliser un algorithme de backtracking
2. Prendre une grille 9x9 (liste de listes, 0 = case vide)
3. Retourner la grille resolue ou None si pas de solution
4. Ne pas utiliser de librairies externes (seulement la bibliotheque standard)

Reponds uniquement avec le code Python de la fonction, sans aucun commentaire ou explication.
"""
    
    response = client.call([
        {"role": "user", "content": prompt}
    ])
    
    # Extraire le code Python
    code = response.strip()
    if "```" in code:
        # Extraire le code entre les backticks
        import re
        match = re.search(r'```python\n(.*?)```', code, re.DOTALL)
        if match:
            code = match.group(1)
    
    # Executer le code
    try:
        # Creer un environnement d'execution sécurisé
        exec_globals = {"__builtins__": {"range": range, "len": len, "list": list, "set": set}}
        exec_locals = {}
        
        exec(code, exec_globals, exec_locals)
        
        if "solve_sudoku" in exec_locals:
            solve_func = exec_locals["solve_sudoku"]
            solution = solve_func(grid)
            return solution
        else:
            print("Erreur: La fonction solve_sudoku n'a pas ete definie.")
            return None
            
    except Exception as e:
        print(f"Erreur lors de l'execution du code genere: {e}")
        return None

# Test code interpreter
print("=== Test Code Interpreter ===\n")
start = time.time()
solution = code_interpreter_solve(example_grid, client)
elapsed = time.time() - start

if solution:
    print("Solution trouvee via le code genere:")
    print_grid(solution)
    
    is_valid = validate_solution(solution, example_grid)
    print(f"\nSolution valide: {is_valid}")
    print(f"Temps: {elapsed:.2f} secondes")
else:
    print("Echec de la resolution par code interpreter.")

## Benchmark : Comparaison des Approches LLM

Comparons les trois approches sur differents niveaux de difficulte.

In [ ]:
def benchmark_llm_approaches(puzzles: List[str], client: LLMClient, limit: int = 3) -> Dict:
    """Compare les differentes approches LLM."""
    results = {
        "zero_shot": {"solved": 0, "total_time": 0, "valid": 0},
        "few_shot": {"solved": 0, "total_time": 0, "valid": 0},
        "code_interpreter": {"solved": 0, "total_time": 0, "valid": 0}
    }
    
    for i, puzzle_str in enumerate(puzzles[:limit]):
        grid = puzzle_to_grid(puzzle_str)
        print(f"\n--- Puzzle {i+1} ---")
        
        # Zero-shot
        print("Testing Zero-shot...", end=" ")
        start = time.time()
        sol = zero_shot_solve(grid, client)
        elapsed = time.time() - start
        results["zero_shot"]["total_time"] += elapsed
        if sol and validate_solution(sol, grid):
            results["zero_shot"]["valid"] += 1
            print(f"OK ({elapsed:.2f}s)")
        else:
            print("ECHEC")
        results["zero_shot"]["solved"] += 1
        
        # Few-shot
        print("Testing Few-shot...", end=" ")
        start = time.time()
        sol = few_shot_solve(grid, client)
        elapsed = time.time() - start
        results["few_shot"]["total_time"] += elapsed
        if sol and validate_solution(sol, grid):
            results["few_shot"]["valid"] += 1
            print(f"OK ({elapsed:.2f}s)")
        else:
            print("ECHEC")
        results["few_shot"]["solved"] += 1
        
        # Code interpreter
        print("Testing Code Interpreter...", end=" ")
        start = time.time()
        sol = code_interpreter_solve(grid, client)
        elapsed = time.time() - start
        results["code_interpreter"]["total_time"] += elapsed
        if sol and validate_solution(sol, grid):
            results["code_interpreter"]["valid"] += 1
            print(f"OK ({elapsed:.2f}s)")
        else:
            print("ECHEC")
        results["code_interpreter"]["solved"] += 1
    
    return results

# Benchmark
print("=== Benchmark LLM Approches ===\n")
print("(Ceci peut prendre plusieurs minutes...)\n")

benchmark_results = benchmark_llm_approaches(easy_puzzles, client, limit=2)

# Afficher les resultats
print("\n" + "="*60)
print("| Approche | Valides / Total | Taux de succes | Temps moyen |")
print("|----------|-----------------|-----------------|-------------|")
for approach, data in benchmark_results.items():
    total = data["solved"]
    valid = data["valid"]
    rate = (valid / total * 100) if total > 0 else 0
    avg_time = data["total_time"] / total if total > 0 else 0
    print(f"| {approach:20} | {valid:3} / {total:3} | {rate:14.1f}% | {avg_time:10.2f}s |")

## Analyse des Resultats

### Observations typiques

1. **Zero-shot** :
   - Reussit sur les puzzles tres faciles (beaucoup d'indices)
   - Echoue souvent sur les puzzles difficiles
   - Parfois produit des solutions invalides (contraintes violees)

2. **Few-shot** :
   - Amelore significativement la performance vs zero-shot
   - Meilleure comprehension du format attendu
   - Echoue encore sur les puzzles difficiles

3. **Code Interpreter** :
   - Taux de succes le plus eleve (>95%)
   - Plus lent (generation + execution du code)
   - Plus fiable car le code peut etre verifie

### LLM vs Solveurs Algorithmiques

| Methode | Taux de succes | Temps (moyen) | Avantages |
|---------|----------------|---------------|-----------|
| **LLM Zero-shot** | ~30-50% | 5-10s | Simple, intuitif |
| **LLM Few-shot** | ~50-70% | 10-20s | Meilleure performance |
| **LLM Code Gen** | ~95-99% | 15-30s | Tres fiable |
| **Backtracking** | 100% | 0.01-0.1s | Rapide, garantie |
| **OR-Tools** | 100% | 0.001-0.01s | Optimal |

### Cas d'usage pour LLM + Sudoku

Malgré leur performance inferieure, les LLM ont des cas d'usage legitimes :

1. **Education** : Expliquer les etapes de resolution
2. **Generation** : Creer de nouveaux puzzles valides
3. **Aide a la resolution** : Suggere les prochaines cases a remplir
4. **Verification** : Verifier si une solution est correcte
5. **Interface naturelle** : Resolution vocale ou textuelle

## Exercices

### Exercice 1 : Ameliorer le Prompt Zero-Shot

Modifier le prompt zero-shot pour inclure des instructions plus specifiques :
- Demander au LLM de verifier sa solution
- Specifier que chaque ligne/colonne/bloc doit contenir 1-9
- Demander de reflechir etape par etape (Chain-of-Thought)

### Exercice 2 : Approche Hybride

Implementer une approche hybride :
1. Utiliser le LLM pour suggerer une case a remplir
2. Verifier que la suggestion est valide
3. Repeter jusqu'a resolution

### Exercice 3 : Generation de Sudokus

Utiliser le LLM pour generer de nouveaux Sudokus :
- Demander de creer une grille complete valide
- Puis demander de retirer des cases pour creer le puzzle
- Verifier que le puzzle a une solution unique

## Conclusion

Dans ce notebook, nous avons explore l'utilisation des **Large Language Models** pour la resolution de Sudoku :

### Points Cles

1. **Trois approches principales** :
   - Zero-shot : Simple mais limite
   - Few-shot : Meilleure performance
   - Code Interpreter : Plus fiable

2. **Performance** :
   - LLM < Solveurs algorithmiques (vitesse et fiabilite)
   - Code Interpreter est l'approche LLM la plus fiable

3. **Cas d'usage** :
   - LLM = Education, interface naturelle, generation
   - Solveurs = Performance, fiabilite, production

### Perspectives

Les LLM s'amelieorent rapidement :
- **GPT-4 (2023)** : ~30-50% succes zero-shot
- **Claude 3.5 (2024)** : ~40-60% succes zero-shot
- **Modeles futurs** : Potentiellement 80-90%+ succes

Cependant, pour les problemes combinatoires comme Sudoku, **les solveurs dedies resteront probablement superieurs** en raison de leur garantie de correction et de leur performance optimale.

### Connexions avec d'autres notebooks

| Notebook | Lien conceptuel |
|----------|-----------------|
| [Sudoku-1-Backtracking](./Sudoku-1-Backtracking.ipynb) | Algorithme que le LLM peut generer |
| [Sudoku-10-ORTools](./Sudoku-10-ORTools.ipynb) | Solveur optimal pour comparaison |
| [Sudoku-16-NeuralNetwork](./Sudoku-16-NeuralNetwork.ipynb) | Autre approche ML (apprentissage supervise) |
| [Sudoku-18-Comparison](./Sudoku-18-Comparison.ipynb) | Comparaison exhaustive de toutes les approches |

---

[< Retour a l'index](./README.md) | **Notebook precedent**: [16 - Neural Network](./Sudoku-16-NeuralNetwork.ipynb) | **Notebook suivant**: [18 - Comparison](./Sudoku-18-Comparison.ipynb)

### References

- **OpenAI (2024)** : "GPT-4 Technical Report"
- **Anthropic (2024)** : "Constitutional AI: Harmlessness from AI Feedback"
- **Kambhampati (2023)** : "On the Planning and Reasoning Capabilities of Large Language Models"
- **Sudoku LLM Solvers** : Plusieurs implementations sur GitHub